#### After run this notebook goto scripts folder and run the evaluation.py script for test the model and see the results in the output folder.


In [11]:
!pip install pandas
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install datasets
!pip install numpy

In [23]:
import pandas as pd
import numpy as np
import os
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

# Ensure output directory exists
os.makedirs("../outputs", exist_ok=True)

# Load and encode dataset
df = pd.read_csv('../data/exceptions.csv')
label_list = sorted(df['label'].unique().tolist())
label_dict = {label: idx for idx, label in enumerate(label_list)}
df['label_encoded'] = df['label'].map(label_dict)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['exception_text'].tolist(),
    df['label_encoded'].tolist(),
    test_size=0.2,
    random_state=42
)

# Tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Dataset
class ExceptionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = ExceptionDataset(train_encodings, train_labels)
val_dataset = ExceptionDataset(val_encodings, val_labels)

# Metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Model & training setup
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_list))

training_args = TrainingArguments(
    output_dir='../models/bert-exception-classifier',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train and save model/tokenizer
train_result = trainer.train()
trainer.save_model('../models/bert-exception-classifier')
tokenizer.save_pretrained('../models/bert-exception-classifier')

# Save training metrics
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
pd.DataFrame([metrics]).to_csv("../outputs/train_metrics.csv", index=False)
print("Training metrics saved to ../outputs/train_metrics.csv")

# 🔍 Final evaluation on validation set
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

val_encodings_eval = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt')
val_encodings_eval = {k: v.to(device) for k, v in val_encodings_eval.items()}

model.eval()
with torch.no_grad():
    outputs = model(**val_encodings_eval)
    preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

val_accuracy = accuracy_score(val_labels, preds)
val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(val_labels, preds, average='weighted', zero_division=0)

# Save final validation metrics
final_metrics_df = pd.DataFrame([{
    "val_accuracy": val_accuracy,
    "val_precision": val_precision,
    "val_recall": val_recall,
    "val_f1": val_f1
}])
final_metrics_df.to_csv("../outputs/final_val_metrics.csv", index=False)
print("Final validation metrics saved to ../outputs/final_val_metrics.csv")

# Print full classification report
print("\nClassification Report on Validation Set:\n")
print(classification_report(val_labels, preds, target_names=label_list))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.379600,0.206152,1.000000,1.000000,1.000000,1.000000
2,0.041400,0.027674,1.000000,1.000000,1.000000,1.000000
3,0.026700,0.019902,1.000000,1.000000,1.000000,1.000000


***** train metrics *****
  epoch                    =        3.0
  total_flos               =    13784GF
  train_loss               =     0.4484
  train_runtime            = 0:00:16.93
  train_samples_per_second =    141.688
  train_steps_per_second   =      8.856
Training metrics saved to ../outputs/train_metrics.csv
Final validation metrics saved to ../outputs/final_val_metrics.csv

Classification Report on Validation Set:

               precision    recall  f1-score   support

  Calibration       1.00      1.00      1.00        20
Contamination       1.00      1.00      1.00        22
    Logistics       1.00      1.00      1.00        27
        Motor       1.00      1.00      1.00        29
     Operator       1.00      1.00      1.00        15
     Pressure       1.00      1.00      1.00        18
   Production       1.00      1.00      1.00        17
       Sensor       1.00      1.00      1.00        18
Specification       1.00      1.00      1.00        22
       System     